In [5]:
import numpy as np
import pandas as pd

df = pd.read_csv('../data/clean_data/movie_level_data.csv')

df = df[['tconst', 'titleType', 'genres', 'averageRating']]

# only keep movies (titleType == 'movie')
df = df[df['titleType'] == 'movie']

# First ensure all values in 'genres' are strings
df['genres'] = df['genres'].astype(str)

# Remove rows with NaN value or empty string for the 'genres' column after conversion
df = df[df['genres'] != 'nan']

# Find the maximum number of genres in any row
max_genres = df['genres'].str.split(',').map(lambda x: len(x) if isinstance(x, list) else 0).max()

# Create new columns for each genre
genre_cols = [f'genre{i+1}' for i in range(max_genres)]

# Split the 'genres' column into separate 'genre' columns
df[genre_cols] = df['genres'].str.split(',', expand=True)

# remove any row if genre1, genre2, or genre3 == #'Biography', 'Sci-Fi', 'Family', 'Documentary', 'History', 'Animation', 'Music', 'War', 'Sport', 'Musical', 'Western', 'Film-Noir', 'News', 'Adult', '\\N'
# First, we will create a set of genres to be removed.
genres_to_remove = {
    'Biography', 'Sci-Fi', 'Family', 'Documentary', 'History',
    'Animation', 'Music', 'War', 'Sport', 'Musical', 'Western',
    'Film-Noir', 'News', 'Adult', '\\N'
}

# Function to determine if a row should be removed
def should_remove(row):
    return any(genre in genres_to_remove for genre in row)

# Filter the DataFrame to exclude rows with genres to be removed
df_filtered = df[~df[genre_cols].apply(should_remove, axis=1)]

# df_filtered now contains only the rows where none of the genres match the genres to remove
df_filtered.reset_index(drop=True, inplace=True)

# remove genre column from df_filtered
df_filtered = df_filtered.drop(columns=['genres'])

# what does the data look like now?
df_filtered.head()

,tconst,titleType,averageRating,genre1,genre2,genre3
0,tt0111161,movie,9.3,Drama,None,None
1,tt0468569,movie,9.0,Action,Crime,Drama
2,tt0137523,movie,8.8,Drama,None,None
3,tt0109830,movie,8.8,Drama,Romance,None
4,tt0110912,movie,8.9,Crime,Drama,None


In [6]:
# Melt the DataFrame so each genre of each movie is in its own row
df_melted = df_filtered.melt(id_vars=['tconst', 'titleType', 'averageRating'], 
                    value_vars=['genre1', 'genre2', 'genre3'],
                    var_name='genre_type', value_name='genre')

# Remove rows where genre is None
df_melted = df_melted[df_melted['genre'].notna()]

# Group by avg rating and genre, then count the number of movies
genre_counts = df_melted.groupby(['averageRating', 'genre']).size().reset_index(name='count')

# Pivot to get years as rows and genres as columns with movie counts as values
df_pivot = genre_counts.pivot(index='averageRating', columns='genre', values='count').fillna(0)

# Convert genres with float counts to int (if necessary after pivot)
for col in df_pivot.columns:
    df_pivot[col] = df_pivot[col].astype(int)

# remove the last row
df_pivot = df_pivot.iloc[:-1]

df_pivot.tail(20)

genre,Action,Adventure,Comedy,Crime,Drama,Fantasy,Horror,Mystery,Romance,Thriller
averageRating,,,,,,,,,,
7.4,68,36,150,112,365,21,29,46,124,52
7.5,82,37,155,120,368,27,26,40,107,71
7.6,59,30,131,95,304,17,15,30,102,49
7.7,51,21,81,62,233,12,5,25,69,56
7.8,45,20,93,50,203,13,10,24,64,32
7.9,30,19,59,44,156,17,8,17,46,28
8.0,35,10,50,41,138,9,9,8,33,30
8.1,27,16,44,31,130,7,4,17,39,32
8.2,22,7,27,29,73,4,1,13,13,21


In [7]:
# Now you can save the DataFrame to JSON
df_pivot.to_json('../data/clean_data/genre_avg_rating.json', orient='index')

In [ ]:
# make a new df out of all the lists
df2 = pd.DataFrame(list(zip(years, Action, Adventure, Comedy, Crime, Drama, Fantasy, Horror, Mystery, Romance, Thriller)),
                columns =['Year', 'Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Thriller'])

df2.head()

In [ ]:
# make Year Index for df2
df2.set_index('Year', inplace=True)

df2.tail()